In [ ]:
import pandas as pd

In [ ]:
table_name = input('Enter table name: ')

In [ ]:
BASE_DIR = f'/Users/itversity/Research/data/retail_db_json/{table_name}'

In [ ]:
import os

In [ ]:
file_name = os.listdir(BASE_DIR)[0]

In [ ]:
file_path = f'{BASE_DIR}/{file_name}'

In [ ]:
df = pd.read_json(file_path, lines=True)

In [ ]:
import boto3

secret_name = "retail.secrets"
region_name = "us-east-1"

session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)

import json

s = json.loads(get_secret_value_response['SecretString'])

s

In [ ]:
get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)

In [ ]:
db_name = 'retail_db'

In [ ]:
conn = f"""postgresql://{s['username']}:{s['password']}@{s['host']}:{s['port']}/{db_name}"""

In [ ]:
df.to_sql(table_name, conn, if_exists='append', index=False)

In [ ]:
import boto3
import json

def get_secrets(secret_name):
    region_name = "us-east-1"

    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )

    return json.loads(get_secret_value_response['SecretString'])

In [ ]:
import os
import pandas as pd

def json_to_df(BASE_DIR, table_name):
    file_name = os.listdir(BASE_DIR.format(table_name=table_name))[0]
    file_path = f'{BASE_DIR.format(table_name=table_name)}/{file_name}'
    return pd.read_json(file_path, lines=True)

In [ ]:
import sys
s = get_secrets('retail.secrets')
db_name = 'retail_db'
conn = f"""postgresql://{s['username']}:{s['password']}@{s['host']}:{s['port']}/{db_name}"""

BASE_DIR = '/Users/itversity/Research/data/retail_db_json/{table_name}'
for table_name in ['departments', 'categories', 'products', 'order_items', 'customers']:
    try:
        df = json_to_df(BASE_DIR, table_name)
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f'{table_name} succesfully populated...')
    except Exception as err:
        print(f'{table_name} failed')
        err_type, err_obj, traceback = sys.exc_info()

        line_num = traceback.tb_lineno
        print ("\npsycopg2 ERROR:", err, "on line number:", line_num)
        print ("psycopg2 traceback:", traceback, "-- type:", err_type)